In [1]:
import pandas as pd

In [2]:
arbitrios_df = pd.read_csv("dataset_vf.csv", sep=";")

/var/folders/m_/_6wk_nxd4xv00ts1kyyz4r3c0000gn/T/ipykernel_20021/430777066.py:1: DtypeWarning: Columns (8,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  arbitrios_df = pd.read_csv("dataset_vf.csv", sep=";")


In [5]:
arbitrios_df.dtypes

FECHA_CORTE                                    int64
PERIODO                                        int64
COD_CONTRIBUYENTE                             object
NOM_CONTRIBUYENTE                             object
COD_PREDIO                                    object
PORCENTAJE_CONDOMINIO                        float64
MONTO_PARQUE_JARDIN                          float64
MONTO_SERENAZGO                               object
MONTO_RESIDUOS_SOLIDOS                        object
MONTO_BARRIDO_CALLES                          object
DESCUENTO_TOPE__PARQUE_JARDIN                float64
DESCUENTO_TOPE_SERENAZGO                      object
DESCUENTO_TOPE_RECOLECCION_RESIDUO_SOLIDO     object
DESCUENTO_TOPE_BARRIDO_CALLE                  object
DEPARTAMENTO                                  object
PROVINCIA                                     object
DISTRITO                                      object
UBIGEO                                       float64
dtype: object

In [6]:
# Función para limpiar números (quitar comas y convertir a float)
def limpiar_numerico(serie):
    if serie.dtype == 'object':
        # Quitar comas y convertir a numeric
        return pd.to_numeric(serie.astype(str).str.replace(',', ''), errors='coerce')
    return serie

# Aplicar conversiones según el diccionario de datos
print("🔄 Convirtiendo tipos de datos según diccionario...")

# Campos numéricos enteros
arbitrios_df['FECHA_CORTE'] = limpiar_numerico(arbitrios_df['FECHA_CORTE']).astype('Int64')
arbitrios_df['PERIODO'] = limpiar_numerico(arbitrios_df['PERIODO']).astype('Int64')

# Campos de texto (ya están bien como object/string)
text_fields = ['COD_CONTRIBUYENTE', 'NOM_CONTRIBUYENTE', 'COD_PREDIO']
for field in text_fields:
    arbitrios_df[field] = arbitrios_df[field].astype('string')

# Campos numéricos decimales (montos y porcentajes)
numeric_fields = [
    'PORCENTAJE_CONDOMINIO',
    'MONTO_PARQUE_JARDIN', 
    'MONTO_SERENAZGO',
    'MONTO_RESIDUO_SOLIDO',
    'MONTO_BARRIDO_CALLE',
    'DESCUENTO_TOPE_PARQUE_JARDIN',
    'DESCUENTO_TOPE_SERENAZGO',
    'MONTO_TOPE_RESIDUO_SOLIDO',
    'DESCUENTO_TOPE_BARRIO_CALLE'
]

for field in numeric_fields:
    if field in arbitrios_df.columns:
        arbitrios_df[field] = limpiar_numerico(arbitrios_df[field])

print("✅ Tipos convertidos!")

🔄 Convirtiendo tipos de datos según diccionario...
✅ Tipos convertidos!


In [7]:
# Verificar los tipos después de la conversión
print("📊 Tipos de datos después de la conversión:")
arbitrios_df.dtypes

📊 Tipos de datos después de la conversión:


FECHA_CORTE                                           Int64
PERIODO                                               Int64
COD_CONTRIBUYENTE                            string[python]
NOM_CONTRIBUYENTE                            string[python]
COD_PREDIO                                   string[python]
PORCENTAJE_CONDOMINIO                               float64
MONTO_PARQUE_JARDIN                                 float64
MONTO_SERENAZGO                                     float64
MONTO_RESIDUOS_SOLIDOS                               object
MONTO_BARRIDO_CALLES                                 object
DESCUENTO_TOPE__PARQUE_JARDIN                       float64
DESCUENTO_TOPE_SERENAZGO                            float64
DESCUENTO_TOPE_RECOLECCION_RESIDUO_SOLIDO            object
DESCUENTO_TOPE_BARRIDO_CALLE                         object
DEPARTAMENTO                                         object
PROVINCIA                                            object
DISTRITO                                

In [10]:
# Función para generar SQL desde DataFrame con tipos correctos
def dataframe_to_sql(df, table_name="df_table", output_file=None):
    """
    Convierte DataFrame a SQL con tipos de datos correctos
    """
    
    if output_file is None:
        output_file = f"{table_name}.sql"
    
    # Mapear tipos de pandas a PostgreSQL
    type_mapping = {
        'Int64': 'INTEGER',
        'int64': 'INTEGER', 
        'float64': 'DECIMAL(15,2)',
        'string': 'VARCHAR(500)',
        'object': 'VARCHAR(500)'
    }
    
    sql_content = []
    
    # CREATE TABLE
    sql_content.append(f"-- Tabla generada desde DataFrame")
    sql_content.append(f"DROP TABLE IF EXISTS {table_name};")
    sql_content.append(f"CREATE TABLE {table_name} (")
    
    columns = []
    for col in df.columns:
        clean_col = col.lower().replace(' ', '_').replace('-', '_')
        pandas_type = str(df[col].dtype)
        
        # Mapear tipos específicos según el diccionario de datos
        if col in ['FECHA_CORTE', 'PERIODO']:
            sql_type = 'INTEGER'
        elif col in ['COD_CONTRIBUYENTE', 'NOM_CONTRIBUYENTE', 'COD_PREDIO']:
            sql_type = 'VARCHAR(500)'
        elif 'MONTO' in col or 'DESCUENTO' in col:
            sql_type = 'DECIMAL(15,2)'
        elif 'PORCENTAJE' in col:
            sql_type = 'DECIMAL(8,2)'
        else:
            sql_type = type_mapping.get(pandas_type, 'VARCHAR(255)')
        
        columns.append(f"    {clean_col} {sql_type}")
    
    sql_content.append(",\n".join(columns))
    sql_content.append(");")
    sql_content.append("")
    
    # INSERT statements
    sql_content.append(f"-- Insertar datos ({len(df)} registros)")
    
    for index, row in df.iterrows():
        values = []
        for col in df.columns:
            value = row[col]
            
            if pd.isna(value) or value is None:
                values.append("NULL")
            elif isinstance(value, str) or df[col].dtype == 'string':
                # Escapar comillas simples y limpiar
                clean_value = str(value).replace("'", "''")
                values.append(f"'{clean_value}'")
            else:
                values.append(str(value))
        
        clean_columns = [col.lower().replace(' ', '_').replace('-', '_') for col in df.columns]
        columns_str = ", ".join(clean_columns)
        values_str = ", ".join(values)
        
        sql_content.append(f"INSERT INTO {table_name} ({columns_str}) VALUES ({values_str});")
        
        # Progreso cada 1000 registros
        if (index + 1) % 1000 == 0:
            print(f"📝 Procesados {index + 1} registros...")
    
    # Escribir archivo
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(sql_content))
    
    print(f"✅ Archivo SQL generado: {output_file}")
    print(f"📊 Registros: {len(df)}")
    print(f"📋 Columnas: {len(df.columns)}")
    
    return output_file

# Generar el SQL
sql_file = dataframe_to_sql(arbitrios_df, "arbitrios_jm", "arbitrios_clean.sql")

📝 Procesados 1000 registros...
📝 Procesados 2000 registros...
📝 Procesados 3000 registros...
📝 Procesados 4000 registros...
📝 Procesados 5000 registros...
📝 Procesados 6000 registros...
📝 Procesados 7000 registros...
📝 Procesados 8000 registros...
📝 Procesados 9000 registros...
📝 Procesados 10000 registros...
📝 Procesados 11000 registros...
📝 Procesados 12000 registros...
📝 Procesados 13000 registros...
📝 Procesados 14000 registros...
📝 Procesados 15000 registros...
📝 Procesados 16000 registros...
📝 Procesados 17000 registros...
📝 Procesados 18000 registros...
📝 Procesados 19000 registros...
📝 Procesados 20000 registros...
📝 Procesados 21000 registros...
📝 Procesados 22000 registros...
📝 Procesados 23000 registros...
📝 Procesados 24000 registros...
📝 Procesados 25000 registros...
📝 Procesados 26000 registros...
📝 Procesados 27000 registros...
📝 Procesados 28000 registros...
📝 Procesados 29000 registros...
📝 Procesados 30000 registros...
📝 Procesados 31000 registros...
📝 Procesados 3200